## 텍스트 파일을 분석하여 네트워크용 파일로 만들기
- 텍스트 파일에서 네트워크 분석을 위한 데이터 추출(노드(node)와 엣지(edge) 생성)
- 동시 출현 단어(word co-occurrence) 네트워크: 한 문서나 문장 내에서 같이 사용되는 단어들의 쌍을 추출하고 빈도를 계산
- 영화 대본의 등장 인물간 관계 네트워크: 같은 씬에서 대화를 주고 받는 인물들은 서로 연관성이 있을 것이라는 전제에서 출발
- **nltk** 라이브러리를 사용한다. **nltk** 라이브러리는 텍스트 자료를 처리하기 위한 다양한 함수들과 모듈들이 내장되어 있다.
- **sklearn** 라이브러리에서 **sklearn.feature_extraction.text** 모듈을 사용한다. 여기에는 텍스트에서 정보를 추출하기 위한 다양한 함수들이 내장되어 있다.

#### Process
1. 동시 출현 단어(word co-occurrence) 네트워크: 한 문서나 문장 내에서 같이 사용되는 단어들은 서로 연관성이 있을 것이라는 전제에서 출발
   - 문서 혹은 문장에서 같이 사용되는 단어들의 쌍을 추출하고 빈도를 계산 후 2 X 2 차원의 데이터프레임을 만듦
   - csv 파일로 내보낸 후 gephi에서 처리
   - 데이터프레임을 gephi용 파일로 만든 후 저장, gephi에서 읽어 처리
1. 영화 대본의 등장 인물간 관계 네트워크: 같은 씬에서 대화를 주고 받는 인물들은 서로 연관성이 있을 것이라는 전제에서 출발
   - 영화 대본을 씬으로 구분 후 각 씬에서 대화를 주고 받는 등장 인물들 이름만 추출
   - 각 씬을 하나의 문서 내지 문장으로 보고 여기서 동시 출현 인물,즉 인물들의 쌍과 빈도를 구해 데이터프레임으로 만듦
   - csv 파일로 내보낸 후 gephi에서 처리
   - 데이터프레임을 gephi용 파일로 만든 후 저장, gephi에서 읽어 처리

#### 1. 동시 출현 단어(word co-occurrence) 네트워크
1. 텍스트(Tesla라는 단어가 들어간 뉴스 기사 제목 100개)를 읽어온다.
1. 텍스트에서 불필요한 기호나 숫자 등을 삭제하고 불용어(stopwords)를 제거하는 등의 텍스트 정제 작업을 실시한다.
1. 각 문서에 나타나는 단어들의 발생 빈도를 구하여 텍스트를 문서-단어(document-term) 행렬로 만든다.
1. 문서-단어(document-term) 행렬을 단어-단어 행렬로 변환하여 두 단어간 동시 발생 빈도를 구한다.
1. 단어-단어 행렬을 데이터프레임으로 변환하여 csv 파일로 저장한다. 그리고 이 행렬을 네트워크 분석 도구인 gephi에서 읽을 수 있는 파일로 저장한다.

##### 텍스트 파일 정제

In [ ]:
with open('../data/news_titles_tesla.txt', 'r') as f:
    news = f.read().splitlines()

print(news[:5])

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

stemmer = PorterStemmer()
english_stopwords = stopwords.words('english')
english_stopwords.append('reuters')

news_cleaned = []
for sent in news:
    tokenized_sent = word_tokenize(text=sent)

    words = []
    for word in tokenized_sent:
        wrd_lower = word.lower()
        if (wrd_lower.isalpha()) and (wrd_lower not in english_stopwords):
            wrd_stem = stemmer.stem(wrd_lower)
            words.append(wrd_stem)
    
    cleaned_title = ' '.join(words)
    news_cleaned.append(cleaned_title)

In [ ]:
print(news_cleaned[:5])

##### 네트워크 노드(node)의 수 줄이기
- 동시 출현 단어 네트워크는 각 단어가 노드(node)가 되고, 이 단어들간의 관계(동일 맥락에서 같이 사용된 단어들)를 엣지(edge)로 표현한다. 너무 많은 단어들을 노드로 사용할 경우 네트워크의 생성에 오랜 시간이 걸리고 확인하기도 어려우므로 노드의 수를 제한한다.
- 아래 코드는 네트워크 그래프의 노드로 사용하기 위해 전체 단어들 중에서 가장 많이 사용된 상위 50개의 단어만 추출하는 단계이다.

In [ ]:
from collections import Counter
import pandas as pd

all_words = []
for title in news_cleaned:
    words = title.split(' ')
    all_words.extend(words)

freqs = Counter(all_words)
most_freq_wrds = freqs.most_common(50)
target_wrds = []
for tup in most_freq_wrds:
    target_wrds.append(tup[0])

In [ ]:
print(target_wrds)

##### 동시 출현 단어(word co-occurrences) 데이터 만들기
- **sklearn** 라이브러리 활용
- 만약 No module named 'sklearn' 오류가 발생하면 이 라이브러리를 설치한다.
- pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(vocabulary=target_wrds)
docTerm = cv.fit_transform(news_cleaned)

docTerm_co = (docTerm.T * docTerm)

In [ ]:
sp_df = pd.DataFrame.sparse.from_spmatrix(docTerm_co, index=target_wrds, columns=target_wrds)
sp_df.iloc[:10, :10]

In [ ]:
docTerm_co.setdiag(0)

In [ ]:
sp_df = pd.DataFrame.sparse.from_spmatrix(docTerm_co, index=target_wrds, columns=target_wrds)
sp_df.iloc[:10, :10]

In [ ]:
sp_df.to_csv('../result/word_cooccurrence_test_reduced_1.csv')

In [ ]:
import networkx as nx

Graph = nx.from_pandas_adjacency(sp_df)
# nx.info(Graph)
nx.write_gexf(Graph, "../result/cooccurrence_test_reduced.gexf")

#### 2. 영화 대본의 등장 인물간 관계 네트워크
- 영화 Love actually의 각 씬(scene)에서 동시에 등장하는 인물들(즉 서로 같은 공간에 있으면서 대화를 나누는 인물들)은 서로 관련이 있다는 것을 전제
- 위에서 설명한 word co-occurrence와 동일한 개념. 여기서는 단어 대신 각 씬에 등장하는 인물 이름간의 관계 즉 인물의 동시 출현(co-occurrence)을 계산
- 다음과 같이 단계로 영화의 등장 인물간 관계 네트워크를 그릴 수 있다.
1. 영화 대본 파일(Love Actually)을 읽어온다.
1. 텍스트에서 불필요한 기호나 숫자 등을 삭제하고 빈 행을 제거하는 등의 텍스트 정제 작업을 실시한다.
1. 전체 문서를 씬(scene)별로 나눈다. 이는 씬 단위로 구분하여 같이 등장하는 인물들을 확인해야 하기 때문이다.
1. 각 씬에서 등장 인물 이름만 남기고 대사와 지문은 삭제한다. 이로써 등장 인물명을 개별 단어처럼 취급할 수 있다.
1. 각 씬에 나타나는 인물들의 등장 빈도를 구하여 씬-인물명(document-term) 행렬로 만든다.
1. 씬-인물명(document-term) 행렬을 인물명-인물명 행렬로 변환하여 두 인물간 동시 등장 빈도를 구한다.
1. 인물명-인물명 행렬을 데이터프레임으로 변환하여 csv 파일로 저장한다. 그리고 이 행렬을 네트워크 분석 도구인 gephi에서 읽을 수 있는 파일로 저장한다.

In [ ]:
with open('../data/love_actually.txt', 'r') as f:
    raw_script = f.readlines()

raw_script[:10]

In [ ]:
script = []
for line in raw_script:
    line = line.strip()
    if line != '':
        script.append(line)

script[:10]

In [ ]:
script[-1]

In [ ]:
script = script[1:-1]
script[:10]

- 불필요한 라인들을 모두 삭제했으므로 이제 대본을 씬 별로 분리하기만 하면 된다.
- 먼저 씬 구분 라인을 찾아서 해당 위치값을 저장하여 이 위치값으로 각 씬의 시작과 끝을 확인한다. 

In [ ]:
import re

scene_idx = []
for i, line in enumerate(script):
    if re.search('\[ Scene #\d{1,2} \]', line):
        scene_idx.append(i)

print(len(scene_idx))

In [ ]:
scene_idx.append(len(script))

In [ ]:
print(scene_idx)

In [ ]:
srt_end_pairs = []
for id_pair in zip(scene_idx[0:], scene_idx[1:]):
    srt = id_pair[0] + 1
    end = id_pair[1]
    srt_end_pairs.append((srt, end))

In [ ]:
scene_texts = []
for id_pair in srt_end_pairs:
    scene = script[id_pair[0]:id_pair[1]]
    scene_texts.append(scene)

In [ ]:
print(scene_texts[1])

- 대본의 등장 인물과 그 인물을 연기한 배우의 이름이 담긴 데이터 파일을 이용하여 대본에 나타나는 등장 인물 이름에 배우 이름을 넣어준다. 

In [ ]:
import pandas as pd

char_cast = pd.read_csv('../data/love_actually_cast.csv')
char_cast.head(3)

In [ ]:
print(char_cast[char_cast['speaker'].str.contains('Billy')]['actor'].to_list()[0])

In [ ]:
chars_per_scene = []
for text in scene_texts:
    chars = []
    for line in text:
        if re.search('^[a-zA-Z]+:', line) is not None:
            x = re.search('^[a-zA-Z]+:', line).group()
            chars.append(x)
    # chars = [re.search('^[a-zA-Z]+:', line).group() for line in text if re.search('^[a-zA-Z]+:', line) is not None]
    chars = [re.sub(':', '', char) for char in chars]
    char_actor = []
    for char in chars:
        if char in char_cast['speaker'].to_list():
            actor = char_cast[char_cast['speaker'].str.contains(char)]['actor'].to_list()[0]
            actor = re.sub(' ', '_', actor)
            char_actor.append(char + '(' + actor + ')')
        # else:
        #     continue
    char_actor_ = ' '.join(char_actor)
    chars_per_scene.append(char_actor_)

In [ ]:
chars = []
for line in scene_texts[1]:
    if re.search('^[a-zA-Z]+:', line) is not None:
        x = re.search('^[a-zA-Z]+:', line).group()
        chars.append(x)

chars_1 = [re.sub(':', '', char) for char in chars]

print(chars_1)

In [ ]:
chars_1 = []
for char in chars:
    x = re.sub(':', '', char)
    chars_1.append(x)

print(chars_1)

- 16행으로 된 위의 코드 블록 중 3-7행도 list comprehension으로 표현할 수 있다.

In [ ]:
text = scene_texts[1]
x = [re.search('^[a-zA-Z]+:', line).group() for line in text if re.search('^[a-zA-Z]+:', line) is not None]
print(x)

In [ ]:
print(len(chars_per_scene))

In [ ]:
chars_per_scene = [scene for scene in chars_per_scene if scene != '']

In [ ]:
chars_per_scene[0]

- 등장 인물 목록 만들기

In [ ]:
all_chars = [char.split(' ') for char in chars_per_scene]
all_chars_ = []
for chars in all_chars:
    all_chars_.extend(chars)

unique_chars = list(set(all_chars_))
print(unique_chars)

In [ ]:
del unique_chars[0]
print(unique_chars)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import networkx as nx

cv = CountVectorizer(lowercase=False, token_pattern='[a-zA-Z]+\([a-zA-Z_]+\)+')
docTerm = cv.fit_transform(chars_per_scene)

docTerm_co = (docTerm.T * docTerm)
docTerm_co.setdiag(0)

names = cv.get_feature_names_out()

df = pd.DataFrame(data=docTerm_co.toarray(), columns=names, index=names)

Graph_LA = nx.from_pandas_adjacency(df)
nx.write_gexf(Graph_LA, "../result/chars_interactions_loveActually_test_2.gexf")

- chars_interactions_loveActually_test_2.gexf 파일을 gephi로 열어서 네트워크를 그리면 아래 그림과 같다.